In [187]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta,time
import os
from scipy.signal import butter, lfilter, welch, periodogram, find_peaks
from scipy.fft import fft, fftfreq, ifft, fftshift, rfft, irfft, ifftshift
import math as M
import json
import pickle
import pandas as pd
import pickle
import matplotlib.dates as mdates
import soundfile as sf
from scipy import signal
import multiprocessing

In [188]:
def axvlines(ax = None, xs = [0, 1], ymin=0, ymax=1, **kwargs):
    ax = ax or plt.gca()
    for x in xs:
        ax.axvline(x, ymin=ymin, ymax=ymax, **kwargs)
        
def setup_graph(title='', x_label='', y_label='', fig_size=None):
    fig = plt.figure()
    if fig_size != None:
        fig.set_size_inches(fig_size[0], fig_size[1])
        ax = fig.add_subplot(111)
        ax.set_title(title)
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        
# the data more uniform (just average among the peaks)
def butter_bandpass(lowcut, highcut, Fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, Fs, order):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order) 
    #The Butterworth filter is an analogue filter design which produces the best output response with no ripple in the pass band 
    # or the stop band resulting in a maximally flat filter response but at the expense of a relatively wide transition band
    # reference: https://www.electronics-tutorials.ws/filter/filter_8.html
    y = lfilter(b, a, data)
    return y  

def _nearest_pow_2(x):

    a = M.pow(2, M.ceil(np.log2(x)))
    b = M.pow(2, M.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b   

In [189]:
phone_name = "PaulA"
path = './data/'
#%cd \"./data/RC0067 Nov 2022/\"
all_files = os.listdir(path)

In [190]:
all_files = [i for i in all_files if i.endswith('.wav') and i.startswith('230122')]
#start = datetime.fromisoformat('2021-11-03T00:06:00')
#start = datetime.fromisoformat('2023-01-23T00:03:00')
#start = datetime.fromisoformat('2023-01-24T00:03:00')
start = datetime.fromisoformat('2023-01-22T22:33:00')
len(all_files)
print(start.strftime('%Y_%m_%d_%H_%M_%S'))

2023_01_22_22_33_00


In [191]:
delta = timedelta(minutes=5)
t = np.array([start + (i-1)*delta for i in range(len(all_files))])
#sunrise = time(hour=7,minute=55,second=0)
#sunset = time(17,50,0)

In [192]:
for j in range(0 , len(all_files)):
  file_name = np.sort(all_files)[j]
  i = sorted(all_files).index(file_name) 
  #i = all_files.index("211103_0220.wav") # Example for the Paper
  t_begin = t[i]

  #filter calculation

  lowcut = 20
  highcut = 20000
  #file_name = all_files[i]
  signal_raw, fs = sf.read(path + file_name)
  # signal_raw: audio data(float 64 [-1, 1]), representing the wave value at certain points,
  # fs: sample rate (how many times getting value every second)***
  signal_raw = np.reshape(signal_raw, (-1, 2)[0]) # guess if there are two channels, making them into n*1 matrix.***
  signal_raw[np.isnan(fs)] = 0
  # FILTER
  signal_filt = butter_bandpass_filter(signal_raw, lowcut, highcut, fs, order=3)
  t_vector = np.linspace(0,signal_raw.shape[0]/fs,signal_raw.shape[0])

  #graph making

  s1 = 0
  s2 = 5*60+1 # the time we care about
  nperseg = int(fs/100)
  noverlap = int(0.5*nperseg)
  nfft = nperseg
  f_sxx, t_sxx, Sxx = signal.spectrogram(signal_filt[int(s1*fs):int(s2*fs)], fs, nfft=nfft , window = 'hann',nperseg = nperseg,noverlap = noverlap,scaling='density')# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html

  vmin = 60
  vmax = 150
  fig, ax = plt.subplots(figsize=(14,10))
  plt.pcolormesh(t_sxx, f_sxx, 10*np.log10(Sxx)+169, shading='nearest',vmin = vmin,vmax = vmax) # some modification to dB
  plt.ylim(0,20000)
  plt.ylabel('Frequency [Hz]')
  plt.xlabel('Time [s]')
  plt.colorbar(ticks=np.arange(0,160,10),label = 'dB')

  # ax2 = ax.twinx()
  # ax2.plot(t_vector[0:int(s2*fs)-int(s1*fs)], ratio_norm[int(s1*fs):int(s2*fs)],'k',linewidth=1) #the black area at the middle**
  # plt.ylim(-0.5,0.2)
  # plt.yticks(np.arange(-0.4, 0.202, step=0.25))

  plt.rcParams.update({'font.size': 38})
  plt.axis(xmin = 0,xmax = 300)
  temp_name = "./data/test/" + t[i].strftime('%Y_%m_%d_%H_%M_%S') + "_" + phone_name + "_high" + ".tiff"
  plt.title(t[i].strftime('%Y_%m_%d_%H_%M_%S') + "_" + phone_name + "_high", size = 15)
  # plt.show()
  plt.savefig(temp_name, bbox_inches = 'tight')
  plt.clf()
  plt.close()

In [ ]:
# lowcut = 2000
# highcut = 12000
# #file_name = all_files[i]
# signal_raw, fs = sf.read(path + file_name)
# # signal_raw: audio data(float 64 [-1, 1]), representing the wave value at certain points,
# # fs: sample rate (how many times getting value every second)***
# print(fs)
# print(signal_raw.shape)
# signal_raw = np.reshape(signal_raw, (-1, 2)[0]) # guess if there are two channels, making them into n*1 matrix.***
# signal_raw[np.isnan(fs)] = 0
# # FILTER
# signal_filt = butter_bandpass_filter(signal_raw, lowcut, highcut, fs, order=3)
# t_vector = np.linspace(0,signal_raw.shape[0]/fs,signal_raw.shape[0])


# Detection algorithm
# signal_filt2 = butter_bandpass_filter(signal_raw, lowcut, highcut, fs, order=3)
# temp = np.abs(signal_filt2).cumsum() # cumulative sum of the elements***
# sta = np.zeros(len(signal_filt2))
# lta = np.zeros(len(signal_filt2))
# # most frequently used sampling rate
# sta_ind = int(44100*2/1000) # 0.005 part short time average***
# lta_ind = int(44100*100/1000) # 0.1 part long time average***
# sta[0:sta_ind] = temp[0:sta_ind]
# sta[sta_ind:] = (temp[sta_ind:]-temp[:-sta_ind]) / sta_ind # the average of changing in sta_ind time*
# lta[0:lta_ind] = temp[0:lta_ind]
# lta[lta_ind:] = (temp[lta_ind:] - temp[:-lta_ind]) / lta_ind # the average of changing in lta_ind time*
# ratio = sta/lta # STA/LTA algorithm (usually used in weak-motion applications that try to record as many seismic events as possible) when the ratio become high, something happened***
# ratio_norm = ratio/np.max(ratio)
# ratio_norm = ratio_norm - np.mean(ratio_norm) # adjusted ratio?**

In [ ]:
# s1 = 0
# s2 = 5*60+1 # the time we care about
# nperseg = int(fs/100)
# noverlap = int(0.5*nperseg)
# nfft = nperseg
# f_sxx, t_sxx, Sxx = signal.spectrogram(signal_filt[int(s1*fs):int(s2*fs)], fs, nfft=nfft , window = 'hann',nperseg = nperseg,noverlap = noverlap,scaling='density')# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html

# vmin = 70
# vmax = 150
# fig, ax = plt.subplots(figsize=(14,10))
# plt.pcolormesh(t_sxx, f_sxx, 10*np.log10(Sxx)+199.4, shading='nearest',vmin = vmin,vmax = vmax) # some modification to dB
# plt.ylim(0,20000)
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [s]')
# plt.colorbar(ticks=np.arange(0,160,10),label = 'dB')

# ax2 = ax.twinx()
# ax2.plot(t_vector[0:int(s2*fs)-int(s1*fs)], ratio_norm[int(s1*fs):int(s2*fs)],'k',linewidth=1) #the black area at the middle**
# plt.ylim(-0.5,0.2)
# plt.yticks(np.arange(-0.4, 0.202, step=0.25))

# plt.rcParams.update({'font.size': 38})
# plt.axis(xmin = 0,xmax = 300)
# temp_name = "./paul_hydrophone_B/Paul Hydrophone B/spectrogram(new)/" + t[i].strftime('%Y_%m_%d_%H_%M_%S') + "_" + phone_name + ".tiff"
# plt.savefig(temp_name,bbox_inches = 'tight')

In [ ]:
# i = 0 

# while i < 5:
#   s1 = int(i*60*fs)
#   s2 = int((i+1)*60*fs)
#   (f, S)= welch(signal_filt[s1:s2], fs)
#   #Power spectral density function (PSD) shows the strength of the variations(energy) as a function of frequency. 
#   #In other words, it shows at which frequencies variations are strong and at which frequencies variations are weak.***
#   print(signal_filt.shape)
#   print(Sxx.shape)
#   plt.semilogy(f, 10*np.log10(S)+199.4)
#   plt.xlim([0, 20000])
#   plt.xlabel('frequency [Hz]')
#   plt.ylabel('Db')
#   plt.show()
#   i = i + 1